### 20 | 单一职责原则：你的模块到底为谁负责？

类的职责是为需求提供解决方案。

Q:类的职责是为需求提供解决方案，这意味着什么？
A:这意味着在一个类的内部函数方法的地位高于成员变量。这牵涉到编写一个类的时候，先确定成员变量，还是先确定函数方法。说实话，我之前编程的时候就是先给类加成员变量。但其实如果把解决需求为第一要务，肯定是先要考虑他的需求应该提供哪些行为。然后，我们根据这些行为提供对应的函数方法，最后才是考虑实现这些方法要有哪些成员变量。所以，这也是在许多编程语言中interface语法里只有方法签名，没有变量定义的原因，成员更多的是作为一种内部实现，设为private。

Q:类的职责是为需求提供解决方案，还意味着什么？
A:类的函数方法的命名应该明确表达自己提供的需求。在一些理论书籍中，形如recevier.message形式的调用方式，称之为消息传递，点操作符左边的receiver是接收者，右边的部分被称为消息。这个消息，我认为就是需求。recevier.message的形式就是让需求在不同的类之间传递，根据设计，有的类不直接处理这个需求把这个需求继续传递下去，有的类则完成这个需求。

比如客户类通过recevier.changePassword()表达他的需求。recevier则接收完成这个需求，或者说，为需求提供解决方案、提供实现。所以，当比较两个函数命名那个更好的时候，是setPassword()更好还是changePassword()更好，我之前觉得这两个命名大差不差没有太大的区分度让我觉得那个更好一些，最多setPassword()更程序员语言一点，changePassword()可读性更好更贴近生活语言，但从哪个更明确地表达了自己的需求，这个角度讲，changePassword()是实打实的用户需求，setPassword()好像是对这个用户需求的翻译。当然这和你在做的软件类型以及你写的代码是不是业务代码有关系，如果你在开发一款操作系统的底层应用，也许setXXX()更能表达所提供的需求。

例子零：提出需求的客户类，应把相似的需求，统一交给某个类，这是高内聚。完成需求的类，应尽量少地与客户类沟通，这是低耦合。这就像领导把任务派给某一岗位的员工，员工不需要领导一天到晚盯盯盯，领导也开心员工也开心。编程的过程总是把关联需求放在一个类里面。类的高耦合性应该贴合需求的高耦合性，所以需求分析是极其重要的。
```
//重构前
class Service {
  public void shutdownTimerTask() {
    // 停止定时器任务
  }
  
  public void shutdownPollTask() {
    // 停止轮询服务
  }
}


class Application {
  private Service service;
  
  public void onShutdown() {
    service.shutdownTimerTask();
    service.shutdownPollTask();
  }
}
```


```
//重构后
class Service {
  private void shutdownTimerTask() {
    // 停止定时器任务
  }
  
  private void shutdownPollTask() {
    // 停止轮询服务
  }
  
  public void shutdown() {
    this.shutdownTimerTask();
    this.shutdownPollTask();
  }
}


class Application {
  private Service service;
  
  public void onShutdown() {
    service.shutdown();
  }
}
```

单一职责原则是朴素的道理，只要成本允许，条线清晰，保持简单，到哪都不会错。
设计程序可以类比成开公司，凡是都有成本，当程序规模本身就很小，就像小公司本身业务就简单，员工身兼数职，不存在管理问题。而随着程序功能的越来越繁杂，我们就希望多加人手，每个人干自己职能范围的事，相应的事只需要找相应的人，出了事也好追责。

单一职责原则，一个类只与一类需求对应，目的是为了当需求或者解决方案变化，使得相关的类需要修改的时候，涉及的类尽可能少，无关的类不动。单一职责很简单，关键是怎么划分职责，这又回到了对需求的分析。

例子一
```

// 一开始
class User {
  // 修改密码
  void changePassword(String password);
  // 加入一个项目
  void joinProject(Project project);
  // 接管一个项目，成为管理员
  void takeOverProject(Project project);
  ...
}

   // 新增需求
  void changePhoneNumber(PhoneNumber phoneNumber):
  
  // 又新增需求
  int countProject();
  
  
  //所以到最后
  
    // 满足管理个人信息的需求
    class PersonalInfoImp {
      // 修改密码
      void changePassword(String password);
      ...
    }

    // 满足参与项目的需求
    class MemberInfoImp{
      // 加入一个项目
      void joinProject(Project project);
      // 接管一个项目，成为管理员
      void takeOverProject(Project project);
      ...
    }
```


例子二
```
//一开始
public class UserInfo {
  private long userId;
  private String username;
  private String email;//身份认证相关
  private String telephone;//身份认证相关
  private long createTime;
  private long lastLoginTime;
  private String avatarUrl;
  private String provinceOfAddress; // 省
  private String cityOfAddress; // 市
  private String regionOfAddress; // 区 
  private String detailedAddress; // 详细地址
  // ...省略其他属性和方法...
}

//专门负责管理地址
```

### 21 | 开放封闭原则：不改代码怎么写新功能？

```
//计算客房价格，因为不同的顾客、不同的客房对应的价格都不同。
//重构前
class HotelService {//酒店类
  public double getRoomPrice(final User user, final Room room) {
    double price = room.getPrice();
    if (user.getLevel() == Level.GOLD) {
      return price * 0.8;
    }
    
    if (user.getLevel() == Level.SILVER) {
      return price * 0.9;
    }
    
    return price;
  }
}
```

```
//重构后
interface UserLevel {
  double getRoomPrice(Room room);
}

class GoldUserLevel implements UserLevel {//酒店SSSVIP接待员类
  public double getRoomPrice(final Room room) {
    return room.getPrice() * 0.8;
  }
}

class SilverUserLevel implements UserLevel {//酒店VIP接待员类
  public double getRoomPrice(final Room room) {
    return room.getPrice() * 0.9;
  }
}


class HotelService {//酒店类
  public double getRoomPrice(final User user, final Room room) {
    return user.getRoomPrice(room);
  }
}

class User {//顾客类
  private UserLevel level;
  ...
  
  public double getRoomPrice(final Room room) {
    return level.getRoomPrice(room);
  }
}
```

Q:为什么把计算客房价格的事情放给酒店类有判断逻辑，而把计算客房价格的事情放给顾客类就没有判断逻辑？
A:其实宏观世界也是这样。只是宏观世界不敢让顾客自己算房价。


甚至可以专门交给不同的顾客等级类GoldUserLevel、SilverUserLevel负责某一顾客等级的计算需求，这是又把计算任务，交还给了酒店里的某个员工，这个员工专门负责接待某一等级的顾客

Q:说实话，原本酒店类自己就能完成的活(计算房价)，依据开闭原则重构之后，变成了酒店问顾客，顾客问接待员一大串流程，颇有官僚衙门之风气，衙门给出这样做的理由是什么？
A:对官僚衙门来说维稳是第一位的，重构后，再增加新的顾客等级，酒店类、顾客类都稳定、不用动，酒店类找顾客类，顾客类找新聘的接待员。

例子二
```
//重构前
class ReportService {
  public void process() {
    // 获取当天的订单
    List<Order> orders = fetchDailyOrders();
    // 生成统计信息
    OrderStatistics statistics = generateOrderStatistics(orders);
    // 生成统计报表
    generateStatisticsReport(statistics);
    // 发送统计邮件
    sendStatisticsByMail(statistics);
  }
}
```

```
//重构后
interface OrderStatisticsConsumer {
  void consume(OrderStatistics statistics);
}

class StatisticsReporter implements OrderStatisticsConsumer {
  public void consume(OrderStatistics statistics) {
    generateStatisticsReport(statistics);
  }
}

class StatisticsByMailer implements OrderStatisticsConsumer {
  public void consume(OrderStatistics statistics) {
    sendStatisticsByMail(statistics);
  }
}

class ReportService {
  private List<OrderStatisticsConsumer> consumers;
  
  void process() {
    // 获取当天的订单
    List<Order> orders = fetchDailyOrders();
    // 生成统计信息
    OrderStatistics statistics = generateOrderStatistics(orders);
    
    for (OrderStatisticsConsumer consumer: consumers) {
        consumer.consume(statistics);
    }
  }
}
```

### 22 | Liskov替换原则：用了继承，子类就设计对了吗？

LSP原则是判定两个类是否构成多态的标准。在Java中，多态很大部分是通过继承加重写实现的。LSP原则告诉我们继承加重写不等于多态，如果一开始就继承错了。

```
class Rectangle {
  private int height;
  private int width;
  
  // 设置长度
  public void setHeight(int height) {
    this.height = height;
  }
  
  // 设置宽度
  public void setWidth(int width) {
    this.width = width;
  }
  
  //
  public void area() {
    return this.height * this.width;
  }
}

class Square extends Rectangle {
  // 设置边长
  public void setSide(int side) {
    this.setHeight(side);
    this.setWidth(side);
t
  }
  
  @Override
  public void setHeight(int height) {
    this.setSide(height);
  }

  @Override
  public void setWidth(int width) {
    this.setSide(width);
  }
}


Rectangle rect = new Square();
rect.setHeight(4); // 设置长度
rect.setWidth(5);  // 设置宽度
assertThat(rect.area(), is(20)); // 对结果进行断言
```
正方形是特殊的长方形，正方形是一种长方形，照这个道理说长方形、正方形满足is-a语义，可以构成继承关系。但程序设计关注的是使用上的可替代性，在宏观世界，正方形、长方形本身就不具有可替换性，所以正方形、长方形不构成继承关系。在程序领域，is-a关系是专有名词，is-a指的是双方互可替代。

为什么正方形、长方形互不可替代？
因为正方形的特殊性，给自己加了父类不拥有的特殊行为，篡改了父类的语义。父类是长方形，子类却限制不能单独设置长和宽，明显违反可替换性。

我们希望程序具有拓展性，而拓展性要求两个类的可替换性，所以才会把强调可替换性的LSP原则列为设计原则。LSP原则下，父类提供接口及其语义，子类必须在符合父类语义的前提下提供实现方法。LSP原则是提醒程序员不要记了接口忘了语义。

```

class Students extends ArrayList<Student> {
  ...
}

```
显然糟糕透了。Students和ArrayList<Student>语义上风马牛不像及。换句话说，如果需要extends ArrayList<Student>，一定是因为

接口语法包含的信息：what和how，what指有什么方法，how指方式的格式，格式决定了我怎么call

### 24 | 依赖倒置原则：高层代码和底层代码，到底谁该依赖谁？

一开始编程的时候，往往会让客户类/上层类依赖下层类，这会导致下层类的修改传导到客户类/上层类，依赖倒置原则说的是在客户类和下层类之间定义接口/接口层，隔绝传导。这就产生了一个局面，客户类和下层类共同依赖于这个接口。因为类是自带接口的，public方法就是接口，虽然public方法作为接口的优先级低于接口语法。


所以就用类自带的接口怎么样，不通过接口语法重写接口怎么样，

```

class Handler {
  private KafkaProducer producer;
  
  void execute() {
    ...
    Message message = ...;
    producer.send(new KafkaRecord<>("topic", message);
    ...
  }
}
```
KafkaProducer在这里是一个接口，还是一个类？是接口。将具体的类作为接口，这个接口，就是伪接口。客户/客户类应该心里清楚他的需求是只有一个服务商可以满足，还是许多服务商的各种解决方案他都不在乎，前者指需要一个伪接口，后者需要一个接口。
★★★客户类只需要提供需求，指明是否需要多厂商提供解决方案。

每个类都有背靠的厂商，每个类的职责都是根据需求提供解决方案。程序架构的角度讲，具体类的地位最低，他们就是干活的，提供解决方案。制定标准制定流程的类，才是当领导的类。DI 容器/IoC 容器是领导之一？。